In [1]:
import random
import json
import torch

In [2]:
from model import NeuralNet
from nltk_functions import bag_of_words, tokenize

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
with open('data.json', 'r') as json_data:
    intents = json.load(json_data)

In [5]:
FILE = "data.pth"
data = torch.load(FILE)

In [6]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

In [7]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=396, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=105, bias=True)
  (relu): ReLU()
)

In [8]:
bot_name = "Lumi"

In [9]:
def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand the question. Can you rephrase it in a different way?"

In [ ]:
if __name__ == "__main__":
    print("Hi, I'm Lumi! (type 'quit' to exit)")
    while True:
        # sentence = "do you use credit cards?"
        sentence = input("You: ")
        if sentence == "quit":
            break

        resp = get_response(sentence)
        print(resp)

Hi, I'm Lumi! (type 'quit' to exit)


You:  Hi


Hi, I'm Lumi! How Can I help you today?


You:  what are features of mobile banking app?


OCBC Mobile Banking app offers a wide range of features including account balance check, fund transfers, bill payments, mobile wallet integration, and more. It provides a convenient and user-friendly banking experience on the go.


...

Yes, we offer specialized accounts for students. To open a student bank account, visit your nearest OCBC branch with your student identification and other necessary documents. Student accounts often come with benefits such as no minimum balance requirement and discounted fees.


You:  i want to waive off my credit card fee


You can apply for an OCBC credit card online through our website or visit your nearest OCBC branch to fill out the application form. Make sure to provide all necessary documents for a smooth application process.


You:  can i refinance my loans?


Loan refinancing involves replacing an existing loan with a new loan that has better terms and conditions. You can refinance various types of loans including home loans, personal loans, and car loans to lower your interest rate, reduce your monthly payments, or consolidate debt. Contact our loan specialists to explore refinancing options tailored to your financial goals.
